# Simplex algorithm for optimizing a characteristic function

# first we have to input characteristic function and constraints

In [9]:
z = []
z =[int(item) for item in input('Enter coeficiant of characteristic function : ').split()]
print(z)

[5, 2, 3, -1, 1]


In [10]:
m = input('Enter the type of problem: (min or max) : ')

In [11]:
number_of_constraints = int(input('Enter the number of constraints: '))

In [12]:
A = [[int(input('Enter coeficiant of x{} in constraint{} : '.format(i+1, c+1))) for i in range(len(z))] for c in range(number_of_constraints)]

In [13]:
Draws = [input('Enter the Draw of constraint{} : '.format(i+1)) for i in range(number_of_constraints)]

In [14]:
RHS = [int(input('Enter Right Hand Side in constraint{} : '.format(i+1))) for i in range(number_of_constraints)]

In [15]:
sign_of_variables = [input('Enter the sign of x{} : (>= / <= / free) '.format(i+1)) for i in range(len(z))]

# Now lets convert form of problem to its standard form

1. convert all of constraints to =
2. for all x in characteristic function =>  x>=0
3. for all b in Right Hand Side b>=0

1)

In [16]:
for i in range(len(Draws)):

  if Draws[i] == ">=":
    # we have to add a (-1 * x_n+1) in that constraint and in characteristic function with 0 coeficiant
    z.append(0)
    for j in range(number_of_constraints):
      if i == j:
        A[j].append(-1)
      else:
        A[j].append(0)

  elif Draws[i] == "<=":
    # we have to add a (1 * x_n+1) in that constraint and in characteristic function with 0 coeficiant
    z.append(0)
    for j in range(number_of_constraints):
      if i == j:
        A[j].append(1)
      else:
        A[j].append(0)

print(A)
print(z)

[[1, 2, 2, 1, 0], [3, 4, 1, 0, 1]]
[5, 2, 3, -1, 1]


2)

In [17]:
for i in range(len(sign_of_variables)):
  if sign_of_variables[i] == 'free':
    z.insert(i+1, -1 * z[i])
    for j in range(number_of_constraints):
        A[j].insert(i+1, -1 * A[j][i])
  elif sign_of_variables[i] == '<=':
    z[i] = -1 * z[i]
    for j in range(number_of_constraints):
      A[j][i] = -1 * A[j][i]

print(z)
print(A)

[5, 2, 3, -1, 1]
[[1, 2, 2, 1, 0], [3, 4, 1, 0, 1]]


3.

In [18]:
for i in range(len(RHS)):
    if RHS[i] < 0:
        for j in range(len(z)):
            A[i][j] = -1 *A[i][j]
        RHS[i] = -1 * RHS[i]

In [19]:
'''
    now we have an standard optimization problem
    lets print the standard form and then solve it by simplex algorithm
'''
print(m, end="  ")
print('z = ', end="")
for i in range(len(z)):
  print(str(z[i]) + 'x.{}'.format(i+1), end=" ")
print()
print('subject to : ')
for i in range(number_of_constraints):
  for j in range(len(z)):
    print(str(A[i][j]) + '.x{}'.format(j+1), end=" ")
  print('=', end=" ")
  print(RHS[i])

for i in range(len(z)):
  print('x.{}'.format(i+1), end=" ")
print('>= 0')

max  z = 5x.1 2x.2 3x.3 -1x.4 1x.5 
subject to : 
1.x1 2.x2 2.x3 1.x4 0.x5 = 8
3.x1 4.x2 1.x3 0.x4 1.x5 = 7
x.1 x.2 x.3 x.4 x.5 >= 0


# simplex method for optimization

first we have to know our problem got the answer with regular simplex method or needs to another simplex method

In [20]:
def reg_simplex(A_simplex, z, number_of_constraints):
  base_counter = 0
  for i in range(number_of_constraints):
    for j in range(len(z)):
      counter = 0
      if A_simplex[i][j] == 1:
        for k in range(number_of_constraints):
          if k != i:
            if A_simplex[k][j] == 0:
              counter += 1
        if counter == number_of_constraints - 1:
          base_counter += 1
  if base_counter == number_of_constraints:
    return True
  else:
    return False

In [21]:
def first_x_b(x_b,number_of_constraints, z, A_simplex):
  counter = 0
  for i in range(number_of_constraints):
    for j in range(len(z)):
      if A_simplex[i][j] == 1:
        for k in range(number_of_constraints):
          if k != i and A_simplex[k][j] == 0:
            counter += 1
        if counter == number_of_constraints - 1:
          x_b.append(j)
        counter = 0
  return x_b

In [22]:
def first_c_b(c_b, x_b, z):
  for i in range(len(x_b)):
    c_b.append(z[x_b[i]])
  return

In [23]:
def first_c_bar(c_bar, c_b, A_simplex):
  for i in range(len(c_bar)):
    for j in range(len(c_b)):
      c_bar[i] -= round(A_simplex[j][i] * c_b[j], 2)
  return

In [24]:
def z_value(c_b, sim_RHS):
  z_val = 0
  for i in range(len(c_b)):
    z_val += c_b[i] * sim_RHS[i]
  return z_val

In [25]:
def check_for_improve(c_bar, x_b, mm):
  if mm == 'min':
    for i in range(len(c_bar)):
      if i != x_b and c_bar[i] < 0:
        return True
    return False

  else:
    for i in range(len(c_bar)):
      if i != x_b and c_bar[i] > 0:
        return True
    return False

In [26]:
def minimum_test(add_to_cb, A_simplex, sim_RHS):
  candidates = []
  for i in range(len(sim_RHS)):
    if A_simplex[i][add_to_cb] <= 0:
      candidates.append(1000)
    else:
      candidates.append(round((sim_RHS[i]/A_simplex[i][add_to_cb]), 2))
  temp = min(candidates)
  for i in range(len(candidates)):
    if candidates[i] == temp:
      candidates.clear()
      return i

In [27]:
def row_reduction(A_simplex, c_bar, i, j, sim_RHS):
  divide = A_simplex[j][i]

  for k in range(len(c_bar)):
    A_simplex[j][k] = round(A_simplex[j][k] / divide, 2)
  sim_RHS[j] = round(sim_RHS[j] / divide, 2)

  for k in range(len(sim_RHS)):
    x = A_simplex[k][i]
    if k != j:
      for t in range(len(c_bar)):
          A_simplex[k][t] = (A_simplex[k][t] - round((x * A_simplex[j][t]), 2))
      sim_RHS[k] = sim_RHS[k] - round((x * sim_RHS[j]), 2)

  return

In [28]:
def new_c_bar(c_bar, z, c_b, A_simplex):
  temp = [0] * len(c_bar)
  for i in range(len(c_bar)):
    for j in range(len(c_b)):
       temp[i] += c_b[j] * A_simplex[j][i]
  for i in range(len(c_bar)):
    c_bar[i] = round(z[i] - temp[i], 2)
  return

In [56]:
def tableau(z, c_b, x_b, A_simplex, sim_RHS, c_bar, z_val):
  print('********************')
  print('c_b',end="\t")
  print('x_b', end="\t")
  for i in range(len(z)):
    print(z[i], end="\t")
  print('RHS')
  for i in range(len(sim_RHS)):
    print(c_b[i],end="\t")
    print(x_b[i],end="\t")
    for j in range(len(z)):
      print(A_simplex[i][j], end="\t")
    print(sim_RHS[i])
  print('C_bar : ', end="\t")
  for i in range(len(z)):
    print(round(c_bar[i], 2), end="\t")
  print('z = ' + str(z_val))
  return

In [57]:
def improve(c_b, x_b, z, c_bar, A_simplex, sim_RHS, z_val, mm):
  if mm == 'min':
    add_to_cb = min(c_bar)
    for i in range(len(c_bar)):
      if c_bar[i] == add_to_cb:
        add_to_cb = i
        break
  else:
    add_to_cb = max(c_bar)
    for i in range(len(c_bar)):
      if c_bar[i] == add_to_cb:
        add_to_cb = i
        break

  j = minimum_test(add_to_cb, A_simplex, sim_RHS)
  x_b[j] = i
  c_b[j] = z[i]
  row_reduction(A_simplex, c_bar, i, j, sim_RHS)
  new_c_bar(c_bar, z, c_b, A_simplex)
  z_val = z_value(c_b, sim_RHS)

  tableau(z, c_b, x_b, A_simplex, sim_RHS, c_bar, z_val)
  return

In [58]:
def simplex(A_simplex,number_of_constraints, z, RHS, mm):
  # first values:
  c_b = []
  x_b = []
  
  c_bar = []

  for i in range(len(z)):
    c_bar.append(z[i])

  first_x_b(x_b, number_of_constraints, z, A_simplex)
  first_c_b(c_b, x_b, z)
  first_c_bar(c_bar, c_b, A_simplex)

  z_val = z_value(c_b, RHS)



  tableau(z, c_b, x_b, A_simplex, RHS, c_bar, z_val)
  # try to improve
  while(check_for_improve(c_bar, x_b, mm)):
    improve(c_b, x_b, z, c_bar, A_simplex, RHS, z_val, mm)
  return


In [59]:
def is_there_basic_var(A_simplex_prime, i, z, RHS):

  for j in range(len(z)):
    counter = 0
    if A_simplex_prime[i][j] == 1:
      for k in range(len(RHS)):
        if k != i and A_simplex_prime[k][j] == 0:
            counter += 1
      if counter == len(RHS) - 1:
        return True
  return False

In [60]:
def artificial_var_twoPhase(A_simplex_prime, RHS, z, w):
  for i in range(len(RHS)):
    if is_there_basic_var(A_simplex_prime, i, z, RHS) is False:
      A_simplex_prime[i].append(1)
      w.append(1)
      for j in range(len(RHS)):
        if j != i:
          A_simplex_prime[j].append(0)
  return

In [61]:
def two_phase(A_simplex, number_of_constraints, z, RHS):
  w = [0] * len(z)
  A_simplex_prime = A_simplex[:][:]
  artificial_var_twoPhase(A_simplex_prime, RHS, z, w)
  
  simplex(A_simplex_prime, number_of_constraints, w, RHS, mm='min')
  
  c_b_new = []
  x_b_new = []
  x_b_new = first_x_b(x_b_new, number_of_constraints, w, A_simplex_prime)
  first_c_b(c_b_new, x_b_new, w)
  
  if z_value(c_b_new, RHS) == 0:
    A_simplex_prime = A_simplex[:][:len(z)]
    simplex(A_simplex_prime, number_of_constraints, z, RHS, m)
  else:
    print('this problem would\'nt solve')
  return

In [62]:
def artificial_var_bigM(A_simplex, RHS, z, m):
    for i in range(len(RHS)):
        if is_there_basic_var(A_simplex, i, z, RHS) is False:
            A_simplex[i].append(1)
            if m == 'min':
                z.append(10000)
            else:
                z.append(-10000)
            for j in range(len(RHS)):
                if j != i :
                    A_simplex[j].append(0)
    return

In [63]:
def BigM(A_simplex, RHS, z, m):
    artificial_var_bigM(A_simplex, RHS, z, m)
    
    simplex(A_simplex, number_of_constraints, z, RHS, m)
    return

In [64]:
if __name__=="__main__":
  A_simplex = A[:][:]
  if reg_simplex(A_simplex, z, number_of_constraints):
    simplex(A_simplex, number_of_constraints, z, RHS, m)
  else:
    kind_of_program = int(input(' you want to get the result in which way?  1.two phase 2.BigM'))
    if kind_of_program == 1:
      two_phase(A_simplex, number_of_constraints, z, RHS)
    else:
      BigM(A_simplex, RHS, z, m)

********************
c_b	x_b	5	2	3	-1	1	RHS
3	2	0.0	0.4	1.0	0.6	-0.2	3.4
5	0	1.0	1.2	0.0	-0.2	0.4	1.2
C_bar : 	0.0	-5.2	0.0	-1.8	-0.4	z = 16.2
